In [39]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [40]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
import pandas as pd
import numpy as np
import jsonpath
import json
import time
import datetime
import subprocess
import xlwings as xw
import telebot
import hmac, base64, struct, hashlib
from openpyxl import Workbook, load_workbook
from openpyxl import formatting, styles
from openpyxl.styles import Color, PatternFill, Font, Border
from PIL import ImageGrab
import pyperclip
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', None) #显示单元格完整信息
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

day = 0
pages_user = 150
pages_fircharge = 60
with open(r'C:\Users\User\Desktop\SEO\SEO代码新 0903到期.txt','r') as f:
    access_token = f.read()
# 启动控制台
print('启动cmd。。。。')
command = 'java -jar C:\\Users\\User\\Desktop\\mason-excel-0.0.1-SNAPSHOT.jar'
process = subprocess.Popen(command, shell=True)

url = 'http://fundmng.bsportsadmin.com/api/manage/fund/withdraw/record/list/history'
session = requests.session()

# 第一次获取token
submit_url = 'http://fundmng.bsportsadmin.com/api/manage/user/admin/login/submit'
header0 = {
    'Accept':'application/json, text/plain, */*',
    # 'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.9',
    'Connection':'keep-alive',
    'Content-Length':'48',
    'Content-Type':'application/x-www-form-urlencoded',
    'Cookie':'admin-uid=690; admin-token=db76bebda5274c80adaadd40bd794f24',
    'Device_id':'1.0',
    'Gl_version':'2.0',
    'Host':'fundmng.bsportsadmin.com',
    'Language':'zh_CN',
    'Origin':'http://fundmng.bsportsadmin.com',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/login',
    'Sign':'2bc4c378817f47731f0adf450a627d19',
    'Some':'header',
    'Systemid':"",
    'Timestamp':'1692415901000',
    'Token':'-1',
    'Uid':'-1',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'Version':'1.0'
}
def get_google_code(secret):
    key = base64.b32decode(secret, True)
    msg = struct.pack(">Q", int(time.time()) // 30)
    google_code = hmac.new(key, msg, hashlib.sha1).digest()
    # 很多网上的代码不可用，就在于这儿，没有chr字符串
    o = ord(chr(google_code[19])) & 15
    # google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    return '%06d' % google_code

# selenium模拟浏览器,并运行jar包，生成今日数据
# 指定文件夹路径
folder_path = r'C:\Users\User\Desktop\SEO\_0816'
# 指定文件名
file_name = '今日数据.xlsx'
# 判断文件是否存在
file_path = os.path.join(folder_path, file_name)
if os.path.exists(file_path):
    os.remove(file_path)

# handless无界面浏览器设置
url_jar = 'http://localhost:9881/swagger-ui/index.html#/Mason%20Excel%20%E4%BA%A7%E7%94%9F/index2UsingPOST'
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
path = r'C:\Users\User\IdeaProjects\project1\Bsport\SEO日报\chromedriver.exe'
chrome_options.binary_location = path

# path = r'C:\Users\User\IdeaProjects\project1\Bsport\SEO日报\chromedriver.exe'
def fun_jar():
    browser = webdriver.Chrome(options=chrome_options)
    browser.get(url_jar)
    time.sleep(3)
    button = browser.find_element(By.XPATH,'//div[@class="try-out"]/button')
    button.click()
    time.sleep(2)
    input = browser.find_elements(By.XPATH,'//input')
    input[0].send_keys((datetime.datetime.now()+datetime.timedelta(days=day)).strftime('%Y%m%d'))
    time.sleep(2)
    outpath = r'C:\Users\User\Desktop\SEO\_0816'+'\\'
    input[2].send_keys(9999)
    time.sleep(2)
    input[4].send_keys(outpath)
    time.sleep(2)
    input[5].send_keys((datetime.datetime.now()+datetime.timedelta(days=day)).strftime('%Y%m%d'))
    textarea = browser.find_element(By.TAG_NAME,'textarea')
    textarea.clear()
    textarea.send_keys(access_token)
    time.sleep(3)
    Execute = browser.find_element(By.XPATH,'//button[@class="btn execute opblock-control__btn"]')
    Execute.click()
    browser.quit()

# 运行模拟浏览器函数
fun_jar()
print('开始运行jar包：',time.strftime('%H:%M',time.localtime()))
time.sleep(300)
print('jar包运行结束: ',time.strftime('%H:%M',time.localtime()))

for i in range(3):
    if not os.path.exists(file_path):
        process.terminate()
        process.wait()
        fun_jar()
        time.sleep(300)
        continue
    break

# 后续采集会员列表，首充记录、数据处理
# 读取运行jar包的数据，及历史数据
data_today = pd.read_excel(r'C:\Users\User\Desktop\SEO\_0816\今日数据.xlsx')
data_2_today = pd.read_excel(r'C:\Users\User\Desktop\SEO\_0816\今日数据.xlsx','趋势分析')
daili = pd.read_excel(r'C:\Users\User\Desktop\SEO\数据+ip历史.xlsx','代理总表')
his_data  = pd.read_excel(r'C:\Users\User\Desktop\SEO\SEO总表(12点+18点).xlsx','数据(12点)_2')

# 采集会员列表和会员存记录
url_fircharge = 'http://fundmng.bsportsadmin.com/api/manage/data/detail/firstRecharge'
url_user = 'http://fundmng.bsportsadmin.com/api/manage/user/maintain/user/list'
session = requests.session()

# 采集首存报表
dic_fir = dict({'会员名':[], '所属代理':[],'注册时间':[], '交易时间':[], '交易类型':[], '币种':[], '金额':[]})
# 昨天开始时间戳
yesterday = datetime.date.today() + datetime.timedelta(days=day)
yesterday_start_time = int(time.mktime(time.strptime(str(yesterday), '%Y-%m-%d')))
# 昨天结束时间戳
yesterday_end_time = int(time.mktime(time.strptime(str(datetime.date.today()+ datetime.timedelta(days=day+1)), '%Y-%m-%d'))) - 1
print(yesterday_start_time)
print(yesterday_end_time)

#---------------------token----------------------
# 采集token
google_code = get_google_code('64ehnxj6yily5bhv23kgb62ozuh6yuu2')
data0 = {
    'username': 'Marquis',
    'password': 'qwer123456',
    'code': google_code
}
session0 = requests.Session()
response0  =session0.post(url=submit_url,data=data0,headers=header0)
response0.encoding = 'utf-8'
obj0 = json.loads(response0.text)
token = obj0['data']['token']

header = {
    'Accept':'application/json, text/plain, */*',
    # 'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.9',
    'Connection':'keep-alive',
    'Content-Length':'75',
    'Content-Type':'application/x-www-form-urlencoded',
    'Cookie':'admin-token=67c8b1bd1b434f898ed8570a860355b8; admin-uid=690',
    'Device_id':'1.0',
    'Gl_version':'2.0',
    'Host':'fundmng.bsportsadmin.com',
    'Language':'zh_CN',
    'Menuid':'100112',
    'Opeartionmenu':'%u62A5%u8868%u67E5%u8BE2-%u4F1A%u5458%u9996%u5B58%u62A5%u8868',
    'Origin':'http://fundmng.bsportsadmin.com',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/system/report-query/report-first-recharge',
    'Sign':'ca83944852acc68fe114cbc65f1e1d22',
    'Some':'header',
    'Systemid':'54',
    'Timestamp':'1692092554000',
    'Token':token,
    'Uid':'690',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Version':'1.0'
}
for page in range(1,pages_fircharge+1):
    data = {
        'page': page,
        'size': 20,
        'tradeType': 0,
        'isFake': 0,
        'stime': yesterday_start_time*1000,
        'etime': yesterday_end_time*1000+999
    }
    response = session.post(url_fircharge,headers=header,data=data,timeout=300)
    response.encoding = 'utf-8'
    obj = json.loads(response.text)

    for i in obj['data']['list']:
        dic_fir['会员名'].append(i['userName'])
        dic_fir['所属代理'].append(i['parentName'])
        dic_fir['注册时间'].append(time.strftime('%Y/%m/%d %H:%M:%S',time.localtime(i['createTime']//1000)))
        dic_fir['交易时间'].append(time.strftime('%Y/%m/%d %H:%M:%S',time.localtime(i['regTime']//1000)))
        dic_fir['交易类型'].append(i['tradeType'])
        dic_fir['币种'].append(i['coinCode'])
        dic_fir['金额'].append(i['amount'])
firChargeUser = pd.DataFrame(dic_fir)
print('会员首存行列：',firChargeUser.shape)

# 采集会员列表
dic_user = dict({'会员账号':[], '姓名':[],'代理':[], '注册时间':[]})
for page in range(1,pages_user+1):
    data2 = {
        'page':page,
        'size':20,
        'userVip':'0,1,2,3,4,5,6,7,8,9,10,11',
        'status':'0,1,2,4',
        'sortType':'3',
        'sortStr':'descend',
        'searchType':'1',
        'channelId':'34',
        'registeredStartDate':yesterday_start_time*1000,
        'registeredEndDate':yesterday_end_time*1000+999,
    }
    response2 = session.post(url_user,headers=header,data=data2,timeout=300)
    response2.encoding = 'utf-8'
    obj2 = json.loads(response2.text)
    for i in obj2['data']['list']:
        dic_user['会员账号'].append(i['username'])
        if i['reallyName'] !='':
            dic_user['姓名'].append(i['reallyName'])
        else:
            dic_user['姓名'].append('--')

        dic_user['代理'].append(i['parentName'])
        dic_user['注册时间'].append(time.strftime('%Y/%m/%d %H:%M:%S',time.localtime(i['registerDate']//1000)))
user = pd.DataFrame(dic_user)
print('用户列表行列:',user.shape)
# 删除测试账号
user = user[~user['会员账号'].str.contains('test')&~user['会员账号'].str.contains('ceshi')]
print('去重后：',user.shape)

启动cmd。。。。
开始运行jar包： 11:12
jar包运行结束:  11:17
1693324800
1693411199
会员首存行列： (117, 7)
用户列表行列: (403, 4)
去重后： (341, 4)


In [42]:
print('开始处理shuju...')
shuju = pd.DataFrame({'人员':['Paddy', 'Tony', 'Max', 'Martin', 'Zed', 'Hugo', 'Aber', 'DK', 'Ben','当日汇总'],
                      '日期':(datetime.datetime.now()+datetime.timedelta(days=day)).strftime('%Y/%m/%d'),
                      '时间':(datetime.datetime.now()+datetime.timedelta(days=day)).strftime('%H:%M'),
                      '发送IP':0,
                      '接收IP':0,
                      '对比昨天(总IP)':0,
                      '对比前3天均值(总IP)':0,
                      '对比前7天均值(总IP)':0,
                      '对比昨天(接收IP)':0,
                      '对比前3天均值(接收IP)':0,
                      '对比前7天均值(接收IP)':0,
                      '对比昨天(总注册)':0,
                      '对比前3天均值(总注册)':0,
                      '对比前7天均值(总注册)':0,
                      '对比昨天(总开户)':0,
                      '对比前3天均值(总开户)':0,
                      '对比前7天均值(总开户)':0})

shuju.set_index('人员',inplace = True)
data_today['IP']=pd.to_numeric(data_today['IP'],errors='coerce').replace(np.nan,0).astype('int64')
grp=data_today.groupby('网站名(domain)').agg({'IP':sum})
try:
    shuju.loc['Paddy','发送IP']=grp.loc['paddy.com','IP']
except:
    shuju.loc['Paddy','发送IP']=0
try:
    shuju.loc['Paddy','接收IP']=grp.loc['paddy.bty','IP']
except:
    shuju.loc['Paddy','接收IP']=0
try:
    shuju.loc['Tony','发送IP']=grp.loc['tonyb.com','IP']/2
except:
    shuju.loc['Tony', '发送IP'] = 0
try:
    shuju.loc['Tony','接收IP']=grp.loc['tony.bty','IP']
except:
    shuju.loc['Tony', '接收IP'] = 0
try:
    shuju.loc['Max','发送IP']=grp.loc['mulu.com','IP']
except:
    shuju.loc['Max', '发送IP'] = 0
try:
    shuju.loc['Max','接收IP']=grp.loc['max.bty','IP']
except:
    shuju.loc['Max', '接收IP'] = 0
try:
    shuju.loc['Martin','发送IP']=grp.loc['redquan.com','IP']
except:
    shuju.loc['Martin', '发送IP'] = 0
try:
    shuju.loc['Martin','接收IP']=grp.loc['martin.bty','IP']
except:
    shuju.loc['Martin','接收IP']=0
try:
    shuju.loc['Zed','发送IP']=grp.loc['zed.com','IP']
except:
    shuju.loc['Zed', '发送IP'] = 0
try:
    shuju.loc['Zed','接收IP']=grp.loc['zed.bty','IP']
except:
    shuju.loc['Zed', '接收IP'] = 0
try:
    shuju.loc['Hugo','发送IP']=grp.loc['hugo.com','IP']
except:
    shuju.loc['Hugo', '发送IP'] = 0
try:
    shuju.loc['Hugo','接收IP']=grp.loc['hugo.bty','IP']
except:
    shuju.loc['Hugo','接收IP']=0

shuju.loc['Aber','发送IP']=grp.loc['aber.com','IP']/2
shuju.loc['Aber','接收IP']=grp.loc['aber.bty','IP']
shuju.loc['DK','发送IP']=grp.loc['dk.com','IP']/2
shuju.loc['DK','接收IP']=grp.loc['dk.bty','IP']
shuju.loc['Ben','发送IP']=grp.loc['ben.com','IP']/2
shuju.loc['Ben','接收IP']=grp.loc['ben.bty','IP']
shuju.loc['当日汇总','发送IP']=shuju['发送IP'].sum()
shuju.loc['当日汇总','接收IP']=shuju['接收IP'].sum()

shuju['日期'] = pd.to_datetime(shuju['日期'])
shuju.insert(1,'人员2',shuju.index)
shuju['人员2']=shuju['人员2'].str.lower()

# 第1次merge前，重置索引
shuju.reset_index(inplace=True)

merge_user = pd.merge(user,daili,how = 'left',left_on='代理',right_on='代理线')
grpSEO = merge_user.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
grpSEO.rename(columns={'seo变化数据团队':'注册'},inplace=True)
grpSEO.reset_index(inplace=True)
grpSEO['人员2'] = grpSEO['seo变化数据团队'].str.lower()
grpSEO.set_index('seo变化数据团队',inplace=True)

shuju=shuju.merge(grpSEO,on='人员2',how='left')

shuju['注册率(%)'] = round(shuju['注册']/shuju['发送IP']*100,2)

merge_charge = pd.merge(firChargeUser,daili,how='left',left_on='所属代理',right_on='代理线')
grpCHARGE = merge_charge.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
grpCHARGE= grpCHARGE.rename(columns={'seo变化数据团队':'开户'})
grpCHARGE.reset_index(inplace=True)
grpCHARGE['seo变化数据团队']=grpCHARGE['seo变化数据团队'].str.lower()
grpCHARGE= grpCHARGE.rename(columns={'seo变化数据团队':'人员2'})
# 第2次merge
shuju = pd.merge(shuju,grpCHARGE,how='left',on='人员2')
shuju['转化率(%)'] = round(shuju['开户']/shuju['注册']*100,2)

#------------
# his_data  = pd.read_csv(r'C:\Users\User\Desktop\SEO\SEO每日更新_814.csv',encoding='gbk')
his_data['日期']= pd.to_datetime(his_data['日期'])
be_data = his_data[his_data['日期']==(shuju['日期'][0]+datetime.timedelta(days=-1))][:-1]

# shuju.sort_index(inplace=True)
shuju.set_index('人员',inplace = True)
shuju.sort_index(inplace=True)
be_data.set_index('人员',inplace=True)
be_data.sort_index(inplace=True)

be3_data = his_data[his_data['日期']>=(shuju['日期'][0]+datetime.timedelta(days=-3))]
be3_data = be3_data.groupby('人员').mean()[:-1]

be7_data = his_data[his_data['日期']>=(shuju['日期'][0]+datetime.timedelta(days=-7))]
be7_data = be7_data.groupby('人员').mean()[:-1]

shuju['对比昨天(总IP)']=shuju['发送IP']-be_data['总IP']
shuju['对比前3天均值(总IP)']= shuju['发送IP']-be3_data['总IP']
shuju['对比前7天均值(总IP)']= shuju['发送IP']-be7_data['总IP']

shuju['对比昨天(接收IP)']=shuju['接收IP']-be_data['接收IP']
shuju['对比前3天均值(接收IP)']= shuju['接收IP']-be3_data['接收IP']
shuju['对比前7天均值(接收IP)']= shuju['接收IP']-be7_data['接收IP']

shuju['对比昨天(总注册)']=shuju['注册']-be_data['注册']
shuju['对比前3天均值(总注册)']= shuju['注册']-be3_data['注册']
shuju['对比前7天均值(总注册)']= shuju['注册']-be7_data['注册']

shuju['对比昨天(总开户)']=shuju['开户']-be_data['开户']
shuju['对比前3天均值(总开户)']= shuju['开户']-be3_data['开户']
shuju['对比前7天均值(总开户)']= shuju['开户']-be7_data['开户']
print(shuju)

开始处理shuju...
               日期     人员2     时间  发送IP  接收IP  对比昨天(总IP)  对比前3天均值(总IP)  \
人员                                                                      
Aber   2023-08-30    aber  11:20  1162   289    -1613.0  -1120.666667   
Ben    2023-08-30     ben  11:20   279    45     -146.0    -64.666667   
DK     2023-08-30      dk  11:20    57   682      -73.0   -106.666667   
Hugo   2023-08-30    hugo  11:20   417     0     -922.0   -667.000000   
Martin 2023-08-30  martin  11:20   593     0     -705.0   -938.000000   
Max    2023-08-30     max  11:20    28    23     -177.0   -128.333333   
Paddy  2023-08-30   paddy  11:20     0     6       -1.0     -0.666667   
Tony   2023-08-30    tony  11:20   224     0     -223.0   -175.333333   
Zed    2023-08-30     zed  11:20  2179    98    -2169.0  -1340.000000   
当日汇总   2023-08-30    当日汇总  11:20  4939  1143        NaN           NaN   

        对比前7天均值(总IP)  对比昨天(接收IP)  对比前3天均值(接收IP)  对比前7天均值(接收IP)  对比昨天(总注册)  \
人员                               

In [43]:
shuju = shuju.iloc[:,:5].join(shuju.iloc[:,-4:]).join(shuju.iloc[:,5:-4])
shuju.fillna(0,inplace=True)

shuju.loc[:,'对比昨天(总IP)':]=shuju.loc[:,'对比昨天(总IP)':].astype('int64')
shuju['注册'] = shuju['注册'].astype('int64')
shuju['开户'] = shuju['开户'].astype('int64')

for i in shuju.iloc[:,5:].columns:
    shuju.loc['当日汇总',i]=sum(shuju[i])

# 重置三个率
shuju.loc['当日汇总','注册率(%)']=round(shuju.loc['当日汇总','注册']/shuju.loc['当日汇总','发送IP']*100,2)
shuju.loc['当日汇总','转化率(%)']=round(shuju.loc['当日汇总','开户']/shuju.loc['当日汇总','注册']*100,2)

shuju.insert(3,'人员',shuju.index)
shuju.drop('人员2',inplace=True,axis=1)
print('shuju处理完成。。。。')
be_data = his_data[his_data['日期']==(shuju['日期'][0]+datetime.timedelta(days=-1))]

shuju处理完成。。。。


In [44]:
be_data
shuju2.columns

,日期,时间,人员,总IP,接收IP,注册,注册率(%),开户,转化率(%),对比昨天(总IP),对比前3天均值(总IP),对比前7天均值(总IP),对比昨天(接收IP),对比前3天均值(接收IP),对比前7天均值(接收IP),对比昨天(总注册),对比前3天均值(总注册),对比前7天均值(总注册),对比昨天(总开户),对比前3天均值(总开户),对比前7天均值(总开户)
40,2023-08-29,19:06:00,Aber,2775,479,27.0,0.97,11.0,40.74,177,920,871,17.0,315.0,351.0,-11,13,16,2,-9,-9
41,2023-08-29,19:06:00,Ben,425,104,3.0,0.71,2.0,66.67,-6,155,139,23.0,76.0,82.0,-1,1,1,-2,-32,-36
42,2023-08-29,19:06:00,DK,130,1304,7.0,5.38,1.0,14.29,22,-4,14,37.0,877.0,981.0,3,0,-2,0,-20,-19
43,2023-08-29,19:06:00,Hugo,1339,0,33.0,2.46,8.0,24.24,82,562,593,0.0,-13.0,-18.0,-2,18,21,-5,-16,-18
44,2023-08-29,19:06:00,Martin,1298,0,32.0,2.47,12.0,37.50,-640,-232,-258,0.0,-25.0,-30.0,-11,15,19,-8,-10,-13
45,2023-08-29,19:06:00,Max,205,177,0.0,0.00,0.0,0.00,35,83,82,15.0,122.0,135.0,0,0,0,0,0,0
46,2023-08-29,19:06:00,Paddy,1,14,0.0,0.00,1.0,0.00,0,0,0,5.0,10.0,11.0,0,0,0,1,1,1
47,2023-08-29,19:06:00,Tony,447,0,11.0,2.46,4.0,36.36,-45,117,107,0.0,-4.0,-5.0,-4,4,5,0,-49,-38
48,2023-08-29,19:06:00,Zed,4348,244,10.0,0.23,2.0,20.00,199,1647,1561,-2.0,156.0,173.0,-3,5,6,-2,-31,-34
49,2023-08-29,19:06:00,当日汇总,10968,2322,123.0,1.12,41.0,33.33,-176,3248,3109,95.0,1514.0,1680.0,-29,56,66,-14,-166,-166


,日期,时间,人员,发送IP,接收IP,注册,注册率(%),开户,转化率(%),对比昨天(总IP),对比前3天均值(总IP),对比前7天均值(总IP),对比昨天(接收IP),对比前3天均值(接收IP),对比前7天均值(接收IP),对比昨天(总注册),对比前3天均值(总注册),对比前7天均值(总注册),对比昨天(总开户),对比前3天均值(总开户),对比前7天均值(总开户)
人员,,,,,,,,,,,,,,,,,,,,,
Aber,2023-08-30,11:20,Aber,1162,289,16,1.38,5,31.25,-1613,-1120,-915,-190,-31,91,-11,-6,2,-6,-15,-13
Ben,2023-08-30,11:20,Ben,279,45,4,1.43,2,50.00,-146,-64,-34,-59,-17,7,1,1,2,0,0,-29
DK,2023-08-30,11:20,DK,57,682,5,8.77,3,60.00,-73,-106,-61,-622,-176,163,-2,0,-4,2,-4,-13
Hugo,2023-08-30,11:20,Hugo,417,0,15,3.60,5,33.33,-922,-667,-447,0,-8,-14,-18,-8,-1,-3,-14,-17
Martin,2023-08-30,11:20,Martin,593,0,10,1.69,5,50.00,-705,-938,-912,0,-13,-24,-22,-15,-6,-7,-14,-17
Max,2023-08-30,11:20,Max,28,23,0,0.00,1,0.00,-177,-128,-111,-154,-90,-45,0,0,0,1,-65,-45
Paddy,2023-08-30,11:20,Paddy,0,6,0,0.00,0,0.00,-1,0,0,-8,-1,1,0,0,0,0,0,0
Tony,2023-08-30,11:20,Tony,224,0,3,1.34,0,0.00,-223,-175,-136,0,-1,-4,-8,-6,-3,0,0,0
Zed,2023-08-30,11:20,Zed,2179,98,8,0.37,4,50.00,-2169,-1340,-919,-146,-69,-7,-2,0,3,2,-16,-25


In [67]:
shuju.loc['Paddy','注册']==0

True

In [58]:
shuju2['转化率(%)']

人员
Aber      31.25
Ben       50.00
DK        60.00
Hugo      33.33
Martin    50.00
Max        0.00
Paddy      0.00
Tony       0.00
Zed       50.00
当日汇总      40.98
Name: 转化率(%), dtype: float64

In [69]:
for name in shuju2.index:
    if shuju2.loc[name,'注册2']==0:
        print(name)


Max
Paddy


In [63]:
shuju2.loc['Max','注册2']
shuju2.loc['Max','开户2']

0

1

In [46]:
shuju2= shuju2.rename(columns={'开户':'开户2','注册':'注册2','接收IP':'接收IP2','对比昨天(总开户)':'开户','对比昨天(总注册)':'注册','对比昨天(接收IP)':'接收IP','对比昨天(总IP)':'总IP'})
shuju2.columns

Index(['日期', '时间', '人员', '发送IP', '接收IP2', '注册2', '注册率(%)', '开户2', '转化率(%)',
       '总IP', '对比前3天均值(总IP)', '对比前7天均值(总IP)', '接收IP', '对比前3天均值(接收IP)',
       '对比前7天均值(接收IP)', '注册', '对比前3天均值(总注册)', '对比前7天均值(总注册)', '开户',
       '对比前3天均值(总开户)', '对比前7天均值(总开户)'],
      dtype='object')

In [49]:
with open(r'C:\Users\User\Desktop\SEO\截图文件\seo_12.txt','w') as f:
    f.write(f'截止今日12点,   注册:  {shuju.loc["当日汇总","注册"]} ,开户:  {shuju.loc["当日汇总","开户"]}，整体'
            f'转化率 : {shuju.loc["当日汇总","转化率(%)"]}%\n')
    f.write(f"对比昨日12点,   注册:  {be_data.loc[be_data['人员']=='当日汇总','注册'].values[0]} ,开户:  {be_data.loc[be_data['人员']=='当日汇总','开户'].values[0]}，整体转化率 : {be_data.loc[be_data['人员']=='当日汇总','转化率(%)'].values[0]}%\n")
    f.write(f'同比昨日变化,   注册:  {shuju.loc["当日汇总","注册"]-be_data.loc[be_data["人员"]=="当日汇总","注册"].values[0]} ,开户:  {shuju.loc["当日汇总","开户"]-be_data.loc[be_data["人员"]=="当日汇总","开户"].values[0]}，整体转化率 : {round(shuju.loc["当日汇总","转化率(%)"]-be_data.loc[be_data["人员"]=="当日汇总","转化率(%)"].values[0],2)}%\n')
    f.write('\n')
    f.write(f'人员下降指标如下：\n')
    for i in range(9):
        f.write(shuju2.iloc[i,:]['人员'])
        f.write(', ')
        f.write(str(list(shuju2.iloc[:,11:].iloc[i,:][shuju2.iloc[:,11:].iloc[i,:]<0].index))+'\n')
    f.write('\n')
    f.write(f'转化率<30%的人员：{str(list(shuju[:-1].loc[shuju[:-1]["转化率(%)"]<30,:]["人员"]))}')

43

48

47

1

10

...

3

2

97

1

35

In [36]:
# 增加行末表头
header_shuju = pd.DataFrame({'日期':'日期',
                             '时间':'时间',
                             '人员':'人员',
                             '发送IP':'总IP',
                             '接收IP':'接收IP',
                             '注册':'注册',
                             '注册率(%)':'注册率(%)',
                             '开户':'开户',
                             '转化率(%)':'转化率(%)',
                             '对比昨天(总IP)':'对比昨天(总IP)',
                             '对比前3天均值(总IP)':'对比前3天均值(总IP)',
                             '对比前7天均值(总IP)':'对比前7天均值(总IP)',
                             '对比昨天(接收IP)':'对比昨天(接收IP)',
                             '对比前3天均值(接收IP)':'对比前3天均值(接收IP)',
                             '对比前7天均值(接收IP)':'对比前7天均值(接收IP)',
                             '对比昨天(总注册)':'对比昨天(总注册)',
                             '对比前3天均值(总注册)':'对比前3天均值(总注册)',
                             '对比前7天均值(总注册)':'对比前7天均值(总注册)',
                             '对比昨天(总开户)':'对比昨天(总开户)',
                             '对比前3天均值(总开户)':'对比前3天均值(总开户)',
                             '对比前7天均值(总开户)':'对比前7天均值(总开户)'},index=[0])
shuju = shuju.append(header_shuju)
app = xw.App(visible=False,add_book=False)
book = app.books.open(r'C:\Users\User\Desktop\SEO\SEO总表(12点+18点).xlsx')

sheet_shuju = book.sheets['数据(12点)_2']
row_shuju = sheet_shuju.used_range.last_cell.row

sheet_shuju['A'+str(row_shuju+1)].options(index=False,header = False).value = shuju
# sheet_ip['A'+str(row_ip+1)].options(index=False,header = False).value = ip_data
book.save()
book.close()

# # 添加条件格式
wb = load_workbook(r'C:\Users\User\Desktop\SEO\SEO总表(12点+18点).xlsx')
ws = wb['数据(12点)_2']
redFill = Font(color='FF0000')
ws.conditional_formatting.add(f'K{row_shuju +1}:U{row_shuju+10}',
                              formatting.rule.CellIsRule(operator='lessThan',
                                                         formula=['0'],
                                                         font=redFill))
wb.save(filename=r'C:\Users\User\Desktop\SEO\SEO总表(12点+18点).xlsx')
wb.close()
# # 保存截图
book2 = app.books.open(r'C:\Users\User\Desktop\SEO\SEO总表(12点+18点).xlsx')
sheet2_shuju = book2.sheets['数据(12点)_2']
# sheet2_ip =  book2.sheets['ip历史']
range_shuju = sheet2_shuju.range(f'A{row_shuju+1}:U{row_shuju+11}')
range_shuju.api.CopyPicture()
img_shuju = ImageGrab.grabclipboard()  # 获取剪贴板的图片数据
img_shuju.save(r'C:\Users\User\Desktop\SEO\截图文件\shuju(12h)-2.png')  # 保存图片
# pyperclip.copy('')
# # 删除行末表头
def delete_row(sheet, row_index):
    range_obj = sheet.range(f'A{row_index}:A{row_index}')
    range_obj.api.EntireRow.Delete()
delete_row(sheet2_shuju,row_shuju+11)
time.sleep(2)
#

book2.save()
book2.close()
app.quit()
# # 发送到群
with open(r'C:\Users\User\Desktop\SEO\截图文件\seo_12.txt','r') as f:
    text = f.read()

bot_DA = telebot.TeleBot("6106076754:AAHjxPSBpyjwpY-lq1iEslUufW46XQvAfr0")
bot_DA.send_photo(-677235937,open(r'C:\Users\User\Desktop\SEO\截图文件\shuju(12h)-2.png','rb'))
bot_DA.send_message(-677235937,text)
bot_DA.stop_polling()
# 查看
print(shuju)

True

                         日期     时间      人员  发送IP  接收IP   注册  注册率(%)  开户  \
Aber    2023-08-29 00:00:00  18:13    Aber  2231   403   27    1.21  11   
Ben     2023-08-29 00:00:00  18:13     Ben   328    90    2    0.61   2   
DK      2023-08-29 00:00:00  18:13      DK   107  1011    5    4.67   1   
Hugo    2023-08-29 00:00:00  18:13    Hugo  1102     0   31    2.81   8   
Martin  2023-08-29 00:00:00  18:13  Martin  1049     0   28    2.67  11   
Max     2023-08-29 00:00:00  18:13     Max   170   145    0     0.0   0   
Paddy   2023-08-29 00:00:00  18:13   Paddy     1    12    0     0.0   1   
Tony    2023-08-29 00:00:00  18:13    Tony   374     0   11    2.94   4   
Zed     2023-08-29 00:00:00  18:13     Zed  3537   207   10    0.28   2   
当日汇总    2023-08-29 00:00:00  18:13    当日汇总  8899  1868  114    1.28  40   
0                        日期     时间      人员   总IP  接收IP   注册  注册率(%)  开户   
0                        日期     时间      人员   总IP  接收IP   注册  注册率(%)  开户   

        转化率(%)  对比昨天(总IP

In [19]:
his_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   日期             40 non-null     datetime64[ns]
 1   时间             40 non-null     object        
 2   人员             40 non-null     object        
 3   总IP            40 non-null     int64         
 4   接收IP           40 non-null     int64         
 5   注册             39 non-null     float64       
 6   注册率(%)         40 non-null     float64       
 7   开户             40 non-null     float64       
 8   转化率(%)         40 non-null     float64       
 9   对比昨天(总IP)      40 non-null     int64         
 10  对比前3天均值(总IP)   40 non-null     int64         
 11  对比前7天均值(总IP)   40 non-null     int64         
 12  对比昨天(接收IP)     10 non-null     float64       
 13  对比前3天均值(接收IP)  10 non-null     float64       
 14  对比前7天均值(接收IP)  10 non-null     float64       
 15  对比昨天(总注册)      40 non-nul